# MLflow end-to-end example

In this example we are going to build a model using `mlflow`, pack and deploy locally using `tempo` (in docker and local kubernetes cluster).

We are are going to use follow the MNIST pytorch example from `mlflow`, check this [link](https://github.com/mlflow/mlflow/tree/master/examples/pytorch/MNIST) for more information.



In this example we will:
  * [Train MNIST Model using mlflow and pytorch](#Train-model)
  * [Create tempo artifacts](#Save-model-environment)
  * [Deploy Locally to Docker](#Deploy-to-Docker)
  * [Deploy Locally to Kubernetes](#Deploy-to-Kubernetes)

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

## Train model

We train MNIST model below:

### Install prerequisites

In [1]:
!pip install mlflow 'torchvision>=0.9.1' torch==1.9.0 pytorch-lightning==1.4.0

In [2]:
!rm -fr /tmp/mlflow

In [3]:
%cd /tmp

/tmp


In [4]:
!git clone https://github.com/mlflow/mlflow.git

Cloning into 'mlflow'...
remote: Enumerating objects: 43208, done.
remote: Counting objects: 100% (1650/1650), done.
remote: Compressing objects: 100% (852/852), done.
remote: Total 43208 (delta 990), reused 1275 (delta 772), pack-reused 41558
Receiving objects: 100% (43208/43208), 38.23 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (33108/33108), done.


### Train model using `mlflow`

In [5]:
%cd mlflow/examples/pytorch/MNIST

/tmp/mlflow/examples/pytorch/MNIST


In [6]:
!mlflow run . --no-conda

2021/09/07 14:29:32 INFO mlflow.projects.utils: === Created directory /tmp/tmpeop3xq68 for downloading remote URIs passed to arguments of type 'path' ===
2021/09/07 14:29:32 INFO mlflow.projects.backend.local: === Running command 'python mnist_autolog_example.py \
  --max_epochs 5 \
  --gpus 0 \
  --accelerator None \
  --batch_size 64 \
  --num_workers 3 \
  --lr 0.001 \
  --es_patience 3 \
  --es_mode min \
  --es_verbose True \
  --es_monitor val_loss
' in run with ID 'f2b157e8c75948d69fe21c3998d656b2' === 
9913344it [00:00, 33006030.84it/s]                                              
Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

29696it [00:00, 107097206.87it/s]                                               
Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

1649664it [00:00, 14839486.03it/s]                                              
Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

5120it [

Epoch 4:  92%|████████▏| 860/939 [00:04<00:00, 197.50it/s, loss=0.0576, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|█████████| 939/939 [00:04<00:00, 196.72it/s, loss=0.0576, v_num=0]
                                                                                Epoch 4, global step 4299: val_loss was not in top 1
Epoch 4: 100%|█████████| 939/939 [00:04<00:00, 196.60it/s, loss=0.0576, v_num=0]
2021/09/07 14:31:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/pytorch_lightning/core/memory.py:203: LightningDeprecationWarning: Argument `mode` in `ModelSummary` is deprecated in v1.4 and will be removed in v1.6. Use `max_depth=-1` to replicate `mode=full` behaviour."
/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called a

In [7]:
!tree -L 1 mlruns/0

mlruns/0
├── f2b157e8c75948d69fe21c3998d656b2
└── meta.yaml

1 directory, 1 file


### Choose test image

In [8]:
from torchvision import datasets

mnist_test = datasets.MNIST('/tmp/data', train=False, download=True)
# change the index below to get a different image for testing
mnist_test = list(mnist_test)[0]
img, category = mnist_test
display(img)
print(category)

/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


7


### Tranform test image to numpy

In [9]:
import numpy as np
img_np = np.asarray(img).reshape((1, 28*28)).astype(np.float32)

## Save model environment

In [10]:
import glob
import os

files = glob.glob("mlruns/0/*/")
files.sort(key=os.path.getmtime)

ARTIFACTS_FOLDER = os.path.join(
    os.getcwd(),
    files[-1],
    "artifacts",
    "model"
)
assert os.path.exists(ARTIFACTS_FOLDER)
print(ARTIFACTS_FOLDER)

/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/f2b157e8c75948d69fe21c3998d656b2/artifacts/model


### Define `tempo` model

In [11]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model


mlflow_tag = "mlflow"

pytorch_mnist_model = Model(
    name="test-pytorch-mnist",
    platform=ModelFramework.MLFlow,
    local_folder=ARTIFACTS_FOLDER,
    # if we deploy to kube, this defines where the model artifacts are stored
    uri="s3://tempo/basic/mnist",
    description="A pytorch MNIST model",
)



Insights Manager not initialised as empty URL provided.


### Save model (environment) using `tempo`

Tempo hides many details required to save the model environment for `mlserver`:
- Add required runtime dependencies
- Create a conda pack `environment.tar.gz`

In [12]:
from tempo.serve.loader import save
save(pytorch_mnist_model)

Solving environment: ...working... done

_openmp_mutex-4.5    | 22 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies: ...working... Ran pip subprocess with arguments:
['/home/sa/miniconda3/envs/tempo-7aba01b5-a70b-4083-93d9-d0e9fd58e671/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/condaenv.2shf2mnj.requirements.txt']
Pip subprocess output:
  Using cached mlflow-1.20.2-py3-none-any.whl (14.6 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached cffi-1.14.6-cp37-cp37m-manylinux1_x86_64.whl (402 kB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached ipython-7.26.0-py3-none-any.whl (786 kB)
  Using cached pyDeprecate-0.3.1-py3-none-any.whl (10 kB)
  Using cached pytorch_lightning-1.4.0-py3

Packing environment at '/home/sa/miniconda3/envs/tempo-7aba01b5-a70b-4083-93d9-d0e9fd58e671' to '/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/f2b157e8c75948d69fe21c3998d656b2/artifacts/model/environment.tar.gz'
[########################################] | 100% Completed |  1min 15.6s

## Package Plan ##

  environment location: /home/sa/miniconda3/envs/tempo-7aba01b5-a70b-4083-93d9-d0e9fd58e671


The following packages will be REMOVED:

  _libgcc_mutex-0.1-conda_forge
  _openmp_mutex-4.5-1_gnu
  ca-certificates-2021.5.30-ha878542_0
  ld_impl_linux-64-2.36.1-hea4e1c9_2
  libffi-3.3-h58526e2_2
  libgcc-ng-11.1.0-hc902ee8_8
  libgomp-11.1.0-hc902ee8_8
  libstdcxx-ng-11.1.0-h56837e0_8
  ncurses-6.2-h58526e2_4
  openssl-1.1.1l-h7f98852_0
  pip-21.2.4-pyhd8ed1ab_0
  python-3.7.9-hffdb5ce_100_cpython
  python_abi-3.7-2_cp37m
  readline-8.1-h46c0cb4_0
  setuptools-57.4.0-py37h89c1867_0
  sqlite-3.36.0-h9cd32fc_0
  tk-8.6.11-h27826a3_1
  wheel-0.37.0-pyhd8ed1ab_1
  xz-5.2.5-h516909a_1
  zlib-1.2


Remove all packages in environment /home/sa/miniconda3/envs/tempo-7aba01b5-a70b-4083-93d9-d0e9fd58e671:



## Deploy to Docker

In [ ]:
from tempo import deploy_local
local_deployed_model = deploy_local(pytorch_mnist_model)

In [ ]:
local_prediction = local_deployed_model.predict(img_np)
print(np.nonzero(local_prediction.flatten() == 0))

In [ ]:
local_deployed_model.undeploy()

## Deploy to Kubernetes

### Prerequisites
 
Create a Kind Kubernetes cluster with Minio and Seldon Core installed using Ansible as described [here](https://tempo.readthedocs.io/en/latest/overview/quickstart.html#kubernetes-cluster-with-seldon-core).

In [13]:
%cd -0

/home/sa/code/tempo/docs/examples/mlflow_e2e


In [27]:
!kubectl apply -f k8s/rbac -n production

secret/minio-secret created
serviceaccount/tempo-pipeline created
role.rbac.authorization.k8s.io/tempo-pipeline created
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding created


### Upload artifacts to minio

In [15]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [ ]:
from tempo.serve.loader import upload
upload(pytorch_mnist_model)

In [28]:
from tempo.serve.metadata import SeldonCoreOptions
runtime_options = SeldonCoreOptions(**{
        "remote_options": {
            "namespace": "production",
            "authSecretName": "minio-secret"
        }
    })

### Deploy to `kind`

In [29]:
from tempo import deploy_remote
remote_deployed_model = deploy_remote(pytorch_mnist_model, options=runtime_options)

KeyboardInterrupt: 

In [26]:
remote_prediction = remote_deployed_model.predict(img_np)
print(np.nonzero(remote_prediction.flatten() == 0))

(array([7]),)


In [30]:
remote_deployed_model.undeploy()

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Tue, 07 Sep 2021 16:17:59 GMT', 'Content-Length': '292'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"seldondeployments.machinelearning.seldon.io \"test-pytorch-mnist\" not found","reason":"NotFound","details":{"name":"test-pytorch-mnist","group":"machinelearning.seldon.io","kind":"seldondeployments"},"code":404}

